# Finding a suitable location for opening a restaurant

## Problem statement
        A restaurants success and failure depends on variety of factors. One of those many factors is its location. The location a restaurant is as important as its quality and taste, besides location of a restaurant is an one-off factor. So it's important that we find a optimal location to run a restaurant. As a data scientist, we are required to properly analyze the location of various neighborhoods in the city and find a proper location to open a restaurant.

## Data sets to be used:
 For the given problem I may use the following data sets,
   - Canada neighborhood data
                            
   - Latitude and Longitude data for the corresponding neighborhood
                            
   - Venue Data

## Description on dataset
        First, the Canada neighborhood data is available in this wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. I will be using a Web scraping tool called BeautifulSoup to scrape the website and develop a data frame. The latitude and longitude dataset is already provided by the IBM team. So i can match the neighborhood in both the dataframes and create a new dataframe. For the venue data, i will be using the foursquare api. I have already created a developer account in the website which gives us access to its venue data. 